# Локализация. Упражнения

В этих упражнениях вам предлагается реализовать простые алгоритмы локализации роботов. На лекции, помимо фильтра частиц, были упомянуты методы триангуляции и альфа-бета фильтр. Давайте их реализуем: обе реализации должны быть сделаны в виде класса, у которого должен быть метод update. Метод update принимает на вход измеренения с камеры о положении ориентиров в сосбтвенной системе координат робота.

В качестве ориентиров будем использовать координаты стоек ворот. Для удобства будем пользоваться упрощенным вариантом, при котором ворота разных цветов. В словаре храняться положения стоек в глобальной системе координат (точка (0, 0) нахождится в центре поля)

In [1]:
landmarks = {
    "blue_posts": [
        [ -4.5, -1.0], [ -4.5, 1.0]
    ],
    "yellow_posts":[
        [ 4.5, -1.0], [ 4.5, 1.0]
    ]
}

Пример данных, которые могут поступать в модуль на одном шаге всей системы. Данные во втором словаре – это данные одометрии с собвственной информацией робота о перемещении за один шаг системы. Для проверки работы придумайте и запишите набор данных с такой же структурой. 

In [48]:
observations = {"yellow_posts":[[4.45, -0.99],[4.49, 1.09]], "blue_posts":[]}
shift = {'shift_x':0.0, 'shift_y':0.01, 'shift_angle':-0.4} 

### 1) Реализовать локализацию триангуляцией (3 балл)
    

In [4]:
class Robot:
  def __init__(self,x=0,y=0,ang=0):
    self.x = x
    self.y = y
    self.ang = ang

In [49]:
from math import sqrt
class localization_triangulation:
    
    #put your code here
    def __init__(self,robot,landmarks,n=100):
      self.robot = robot
      self.landmarks = landmarks
      self.n = n
    
    def returnCoo(self):
      return self.robot.x, self.robot.y, self. robot.ang
    
    def get_points_of_intersection(self,x10, y10, r1 , x20, y20, r2):
      d = sqrt((x10-x20)**2 + (y10-y20)**2)
      if d > r1+r2 : return 0;
      a = (r1*r1 - r2*r2 + d*d ) / (2*d)
      h = sqrt(r1**2 - a**2)
      x0 = x10 + a*(x20 - x10)/d;
      y0 = y10 + a*(y20 - y10)/d;
      x1= x0 + h*(y20 - y10)/d;
      y1= y0 - h*(x20 - x10)/d;
      print('First intersection : ', x1,y1)
      if a == r1 : return [x1,y1]
      x2 = x0 - h*(y20 - y10)/d;
      y2 = y0 + h*(x20 - x10)/d;
      print('Second intersection : ', x2,y2)
      return [x1,y1,x2,y2]
    
    def update(self,observations):
      if observations['yellow_posts']:
        dxl = observations['yellow_posts'][0][0]
        dyl = observations['yellow_posts'][0][1]
        r1 = sqrt(dxl**2+dyl**2)
        dxR = observations['yellow_posts'][1][0]
        dyR = observations['yellow_posts'][1][1]
        r2 = sqrt(dxR**2+dyR**2)
        x10 = landmarks['yellow_posts'][0][0]
        y10 = landmarks['yellow_posts'][0][1]
        x20 = landmarks['yellow_posts'][1][0]
        y20 = landmarks['yellow_posts'][1][1]
      else :
        dxl = observations['blue_posts'][0][0]
        dyl = observations['blue_posts'][0][1]
        r1 = sqrt(dxl**2+dyl**2)
        dxR = observations['blue_posts'][1][0]
        dyR = observations['blue_posts'][1][1]
        r2 = sqrt(dxR**2+dyR**2)
        x10 = landmarks['blue_posts'][0][0]
        y10 = landmarks['blue_posts'][0][1]
        x20 = landmarks['blue_posts'][1][0]
        y20 = landmarks['blue_posts'][1][1]
      inter = self.get_points_of_intersection(x10, y10, r1 , x20, y20, r2)
      self.robot.x = inter[2]
      self.robot.y = inter[3]

In [50]:
triang = localization_triangulation(Robot(0.0,0.0,0.0), landmarks)

In [51]:
triang.update(observations)

First intersection :  8.97720962654196 -0.14139999999999997
Second intersection :  0.022790373458040136 -0.14139999999999997


In [52]:
triang.returnCoo()

(0.022790373458040136, -0.14139999999999997, 0.0)

### 2) Дополнить предыдущий пункт до альфа-бета фильтра (5 балла)

In [ ]:
class localization_alphabeta:
    
    #put your code here
    
    pass
    
    def update():
        
        #put your code here
        
        pass

### 3) *Дополнительно* Добавьте визуализацию вида сверху  (2 балла)

Например на OpenCV или Matplotlib, например так как было в семинаре, чтобы можно было удобно посмотреть на работу модуля.


In [3]:
class Field:
  def __init__(self, field_param):
    self.field = field_param
    self.w_width = field_param['main_rectangle'][0][0]
    self.w_lenght = field_param['main_rectangle'][0][0]